In [2]:
%pip install -q -U torch==2.0.1 bitsandbytes==0.40.2
%pip install -q -U transformers==4.35.2 peft==0.4.0 accelerate==0.21.0
%pip install -q -U datasets py7zr einops tensorboardX
!pip install evaluate
# Add installed cuda runtime to path for bitsandbytes
import os
import nvidia

cuda_install_dir = '/'.join(nvidia.__file__.split('/')[:-1]) + '/cuda_runtime/lib/'
os.environ['LD_LIBRARY_PATH'] =  cuda_install_dir


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import pandas as pd
import os
import torch
import numpy as np
# Path to your env.txt file
env_file_path = 'data/env.txt'

# Read and set environment variables
with open(env_file_path, 'r') as file:
    for line in file:
        key, value = line.strip().split('=')
        os.environ[key] = value
token = os.environ['huggingface_token']

## Check Length of MATHWELL Training Data and Prompts

In [4]:
df = pd.read_csv('data/mathwell_generations.csv')
model_path = "meta-llama/Llama-2-70b-hf"   # Specify the path to the model
tokenizer = AutoTokenizer.from_pretrained(model_path, token=token)

In [5]:
question_lengths = 0
for i in range(0, len(df)):
    output = df.iloc[i]['output']
    try: 
        inputs = tokenizer.encode(output, return_tensors="pt")
    except:
        pass
    length = inputs.shape[1]
    question_lengths+=length
print(f"Average question length in tokens: {question_lengths/len(df)}")

Average question length in tokens: 191.32467532467533


In [6]:
df = pd.read_csv('data/mathwell.csv')
import random
topics = ['Superman', "Batman", "Wonder Woman", "Barbie", "Power Rangers", "basketball", "soccer", "football", "volleyball", 'field hockey',\
'Fortnite', 'Spiderman', "Iron Man", "Captain America", "Captain Marvel", "Thor, the God of Thunder", "Ninja Turtles", "Black Panther", "Taylor Swift", "swimming",\
"Pokémon", "Super Mario", "Naruto", "unicorns", "Hello Kitty", "Minecraft", "lacrosse", "cheer leading", "LeBron James", "Steph Curry", "Patrick Mahomes",\
"Serena Williams", "dogs", "cats", "dinosaurs", "Harry Potter", "cars", "planes", "trains", "pizza", "cookies", "ice cream", 'candy']
question_lengths = 0
for i in range(0,1000):
    topic = random.choice(topics)
    final_prompt = f"Write a grade school math word problem about {topic} and Python function with a commented out step-by-step solution to solve the word problem."
    prompt = "Write a grade school math word problem and Python function with a commented out step-by-step solution to solve the word problem."
    questions = []
    for i in range(0, 8):
        question = df['output'].iloc[random.randint(0,len(df)-1)]
        questions.append(question)
    formatted_prompt = []
    for i in range(0,8):
        formatted_prompt.append((f"Below is an instruction that describes a task. "
                f"Write a response that appropriately completes the request.\n\n"
                f"### Instruction:\n{prompt}\n\n### Response: {questions[i]}"))
    formatted_prompt.append(f"Below is an instruction that describes a task. "
                f"Write a response that appropriately completes the request.\n\n"
                f"### Instruction:\n{final_prompt}\n\n### Response: ")
    formatted_prompt = "\n".join(formatted_prompt)
    inputs = tokenizer.encode(formatted_prompt, return_tensors="pt")
    length = inputs.shape[1]
    question_lengths+=length
print(f"Average input length in tokens: {question_lengths/1000}")

Average input length in tokens: 2049.763


In [26]:
def check_length(df):
    question_lengths = 0
    for i in range(0, len(df)):
        output = df.iloc[i]['text']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths+=length
    return f"Average overall question length in tokens: {question_lengths/len(df)}"
def check_mathwell(df):
    question_lengths = 0
    for i in range(0, len(df)):
        output = df.iloc[i]['question']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths+=length
    return f"Average overall question length in tokens: {question_lengths/len(df)}"
def check_length_good(df):
    question_lengths = 0
    for i in range(0, len(df)):
        output = df.iloc[i]['question']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths+=length
    return f"Average good question length in tokens: {question_lengths/len(df)}"
def check_gsm8k(df):
    question_lengths = 0
    for i in range(0, len(df)):
        output = df.iloc[i]['instruction']
        try: 
            inputs = tokenizer.encode(output, return_tensors="pt")
        except:
            pass
        length = inputs.shape[1]
        question_lengths+=length
    return f"Average overall question length in tokens: {question_lengths/len(df)}"

## Check Average Length of GSM8K Training Questions

In [27]:
gsm8k = pd.read_csv('data/gsm8k_questions.csv')
check_gsm8k(gsm8k)

'Average overall question length in tokens: 65.66239462682475'

## Check Length of MATHWELL Training Data, Questions Only

In [25]:
mathwell_all = pd.read_csv('data/mathwell_questions.csv')
check_mathwell(mathwell_all)

'Average overall question length in tokens: 59.23747680890538'

In [10]:
mathwell_train = pd.read_csv('data/mathwell_good_questions.csv')
check_length_good(mathwell_train)

'Average good question length in tokens: 57.527806925498425'

## Check Length of MATHWELL Final Questions

In [11]:
mathwell = pd.read_csv('data/mathwell_solvability.csv')
check_length(mathwell)

'Average overall question length in tokens: 55.24'

In [12]:
mathwell = pd.read_csv('data/mathwell_good.csv')
check_length_good(mathwell)

'Average good question length in tokens: 54.537234042553195'

## Check Length of Llama Questions

In [13]:
llama = pd.read_csv('data/llama_solvability.csv')
check_length(llama)

'Average overall question length in tokens: 53.08'

In [14]:
llama = pd.read_csv('data/llama_good.csv')
check_length_good(llama)

'Average good question length in tokens: 51.547770700636946'

## Check Length of LLEMA Questions

In [15]:
llema = pd.read_csv('data/llema_solvability.csv')
check_length(llema)

'Average overall question length in tokens: 56.516'

In [16]:
llema = pd.read_csv('data/llema_good.csv')
check_length_good(llema)

'Average good question length in tokens: 51.1219512195122'

## Check Length of MAmmoTH Questions

In [17]:
mammoth = pd.read_csv('data/mammoth_solvability.csv')
check_length(mammoth)

'Average overall question length in tokens: 46.056'

In [18]:
mammoth = pd.read_csv('data/mammoth_good.csv')
check_length_good(mammoth)

'Average good question length in tokens: 43.957142857142856'